In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import dask.dataframe as dd
# Caricare il dataset
df3 = pd.read_csv('df_bayesiana3.csv')

print(df3)
df3 = df3.drop(columns = ['Unnamed: 0'])

df3.replace({7: -1, 9: -1, 77 : -1, 99 : -1, 777: -1, 999: -1, 7777: -1, 9999: -1 ,'nan': -1}, inplace=True)
df3.replace(-1.0, np.nan, inplace=True)
# Contare i NaN per colonna
nan_counts = df3.isna().sum()
nan_count_col1 = df3['DIABTYPE'].isna().sum()
print(f'Numero di NaN in col1: {nan_count_col1}')
# Determinare la soglia
soglia = 120000  # Modificare questo valore in base alla soglia desiderata

# Se il numero di valori mancanti in una colonna è maggiore della soglia, eliminare la colonna
df_cleaned = df3.drop(columns=nan_counts[nan_counts > soglia].index)
df_cleaned = df_cleaned.dropna(thresh=df_cleaned.shape[1] - 0)
df_sampled = df_cleaned.astype({col: 'float32' for col in df_cleaned.select_dtypes(include='float64').columns})

#df_sampled = df_cleaned.replace(np.nan, -1.0)
print(df_sampled)
df_sampled = df_sampled.drop(columns = ['SMOKE100','INCOME3','_AGEG5YR.1', 'EDUCA'])
print(df_sampled.dtypes)

train_data, test_data = train_test_split(df_sampled, test_size=0.2, random_state=42)

# Reset indici
df_test_data = test_data.reset_index(drop=True)
df_test_data

In [ ]:
# Previsione sul test set
y_true = test_data['DIABETE4'].values
y_pred = []

In [ ]:
import os
BLOCKS_DIR = "C:\\Users\\CasaLab1\\Desktop\\diabete_gruppo4\\diabete"

files=[os.path.join(BLOCKS_DIR,x) for x in os.listdir(BLOCKS_DIR) if "previsioni_blocco" in x]
files.sort(key=lambda x:int(x.split("_")[-1].split(".")[0]))
for f in files:
    with open(f,"r") as fp:
        y_pred.extend(eval(x.strip()) for x in fp.read().split("\n") if len(x)>0)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Rimuovere None dai predetti e dai veri valori corrispondenti
y_true_filtered = [yt for yt, yp in zip(y_true, y_pred) if yp is not None]
y_pred_filtered = [yp for yp in y_pred if yp is not None]

# Calcolare le metriche di valutazione
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
precision = precision_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted' a seconda del tuo caso
recall = recall_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'
f1 = f1_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
